# Multidimensional PDFs

This tutorial is about handling multiple dimensions when creating a custom PDF.

The differences are marginal since the ordering is handled automatically. It is on the other hand crucial to understand the concept of a `Space`, most notably `obs` and `axes`.

A user (1someone who instantiates the PDF) only knows and handles observables. The relative order does not matter, if a data has observables a and b and a pdf has observables b and a, the data will be reordered automatically.  Inside a PDF on the other hand, we do not care at all about observables but only about the ordering of the data, the *axis*. So any data tensor we have, and limits for integration, normalization etc. **inside** the PDF is order based and uses *axes*. 

When passing the observables to the init of the PDF (as a user), each observable is automatically assigned to an axis corresponding to the order of the observable. The crucial point is therefore to communicate to the user which *axis* corresponds to what. The naming of the observables is completely up to the user, but the order of the observables depends on the pdf. Therefore, the correspondance of each axis to it's meaning has to be stated in the docs.

In [1]:
import zfit
from zfit import ztf
import numpy as np

## Axes, not obs

Since we create a pdf here, we now can completely forget about observables. We can assume that all the data is axes based (order based).We simply need to write down what each axis means.

An example pdf is implemented below. It calculates the lenght of a vector shifted by some number (dummy example).

In [2]:
class AbsVectorShifted(zfit.pdf.ZPDF):
    _N_OBS = 3  # dimension, can be omitted
    _PARAMS = ['xshift', 'yshift']  # the name of the parameters
    
    def _unnormalized_pdf(self, x):
        x, y, z = ztf.unstack_x(x)  # returns a list with the columns: do x, y, z = ztf.unstack_x(x) for 3D
        xshift = self.params['xshift']
        yshift = self.params['yshift']
        x = x + xshift
        y = y + xshift
        return ztf.sqrt(ztf.square(x) + ztf.square(y) + ztf.square(z))  # dummy calculations
        

Done. Now we can use our pdf already!

In [3]:
lower = ((-3, -2, -1),)
upper = ((3, 2, 1),)
obs = zfit.Space(['xobs', "yobs", "z"], limits=(lower, upper))

data_np = np.random.random(size=(1000, 3))
data = zfit.data.Data.from_numpy(array=data_np, obs=obs)  # we could also use a different order of obs here

Create two parameters and an instance of your own pdf

In [4]:
xshift = zfit.Parameter("xshift", 1.)
yshift = zfit.Parameter("yshift", 2.)
abs_vector = AbsVectorShifted(obs=obs, xshift=xshift, yshift=yshift)

Instructions for updating:
Colocations handled automatically by placer.


In [6]:
probs = abs_vector.pdf(data)

In [7]:
probs_np = zfit.run(probs)
print(probs_np[:20])

[0.02298816 0.01838631 0.02237864 0.02185262 0.02294781 0.02005665
 0.01344928 0.01918465 0.01887062 0.02097156 0.02135591 0.0204704
 0.02511017 0.01959294 0.01899909 0.02229129 0.02258128 0.02183022
 0.01827738 0.01791268]


We could improve our PDF by registering an integral. This requires a few steps:
 - define our integral as a function in python
 - define in which space our integral is valid, e.g. whether it is an integral over all axis or only partial and whether any limit is valid or only special (e.g. from -inf to inf)
 - register the integral and say if it supports additional things (e.g. norm_range)
 
Let's start defining the function. This takes, for an integral over all axes, three parameters:
 - limits: the actual limits the integral is over
 - params: the parameters of the model (which _may_ be needed)
 - model: the model (pdf/func) itself
 
we need to calculate the integral and return (currently) a scalar.

In [13]:
def abs_vector_integral_from_any_to_any(limits, params, model):
    lower, upper = limits.limits
    # write your integral here
    return 42.  # dummy integral, must be a scalar!

Now let's define the limits. We want to allow an integral over whole space in three dims, this may looks cumbersome but is straightforward (and done only once):

In [14]:
limits_to_integrate = (((zfit.Space.ANY_LOWER, zfit.Space.ANY_LOWER, zfit.Space.ANY_LOWER),),
                       ((zfit.Space.ANY_UPPER,zfit.Space.ANY_UPPER,zfit.Space.ANY_UPPER),))

Now we need the axis we will integrate over

In [15]:
axes_to_integrate = (0, 1, 2)  # implies this is over all axes of the pdf

Now we create our space and register the integral. In order to change precedency of integrals (e.g. because some are very simple and return a single number, so this special cases should be regarded first), a priority argument can be given. Also if the integral supports multiple limits or norm range calculation, this can be specified here. Otherwise, this is automatically handled and the integral never gets multiple limits resp a norm range (that's why we don't have it in the API of the integral function).

In [16]:
limits = zfit.Space.from_axes(axes=axes_to_integrate, limits=limits_to_integrate)

AbsVectorShifted.register_analytic_integral(func=abs_vector_integral_from_any_to_any, limits=limits,
                                           priority=51,
                                            supports_norm_range=False,  # False by default, but could be set to
                                            supports_multiple_limits=False)  # True. False -> autohandled

### n-dimensional

## Advanced Custom PDF

Subclass BasePDF. The `_unnormalized_pdf` has to be overriden and, in addition, the `__init__`.

Any of the public main methods (`pdf`, `integrate`, `partial_integrate` etc.) can **always** be overriden by implementing the function with a leading underscore, e.g. implement `_pdf` to directly controls `pdf`, the API is the same as the public function without the name. In case, during execution of your own method, it is found to be a bad idea to have overridden the default methods, throwing a `NotImplementedError` will restore the default behavior.

In [ ]:
# TOBEDONE